In [65]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
import numpy as np
from PIL import Image
import glob
import os
from sklearn.model_selection import train_test_split
from math import log, exp

In [66]:
class WL:
    def __init__(self, model):
        self.model = model
        
    def train(self, X, Y):
        self.model.fit(X,Y)
    
    def predict(self,X):
        return self.model.predict(X)

In [67]:
def Boost(X, Y, wl, T):
    D = [1/X.shape[0] for i in range(X.shape[0])]
    w = []
    h = []
    e = []
    for t in range(T):
        h.append(wl.predict(X))
        e.append(sum([D[i] for i in range(X.shape[0]) if h[-1][i]==Y[i]]))
        w.append(1/2*log(1/e[-1]-1))
        old_D = D.copy()
        denom = sum([old_D[j] * exp(-w[-1]*Y[j]*h[-1][j]) for j in range(X.shape[0])])
        D = [(old_D[i]*exp(-w[-1]*Y[i]*h[-1][i]))/denom for i in range(X.shape[0])]
    val = sum([np.array(w[t]) @ np.array(h[t]) for t in range(T)])
    if val > 0:
        return 1
    if val < 0:
        return -1
    return 0

In [68]:
wl = WL(DecisionTreeClassifier(max_depth=2))

In [72]:
IMAGE_DIR = "./data/data/data"

def load():
    file_list = glob.glob(IMAGE_DIR + "/*.jpg")
    X = []
    Y = []

    for fname in file_list:
        with Image.open(fname) as img:
            np_img = np.array(img).flatten()
        label = int(os.path.split(fname)[-1].split('.')[0].split('_')[3])-1

        X.append(np_img)
        tempy = np.zeros(15)
        tempy[label] = 1
        Y.append(tempy)
    X, Y = np.array(X), np.array(Y)
    return(X, Y)

In [74]:
# create test, train split
X, Y = load()
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, train_size=0.8, random_state=2021)
Ytrain_classes = [[1 if np.where(d == 1)[0][0] == i else -1 for d in Ytrain] for i in range(Ytrain.shape[1])]
Ytest_classes = [[1 if np.where(d == 1)[0][0] == i else -1 for d in Ytest] for i in range(Ytest.shape[1])]

[]
[]


ValueError: With n_samples=0, test_size=None and train_size=0.8, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [71]:
for i in range(len(Ytrain_classes)):
    wl.train(Xtrain, Ytrain_classes[i])
    result = Boost(Xtest, Ytest_classes[i], wl, 100)
    print(result)

ValueError: math domain error